In [ ]:
!pip install langchain-community transformers torch sentence-transformers gradio pandas requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import gradio as gr
from langchain_community.llms import HuggingFacePipeline
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langgraph.graph import StateGraph, END, START
from typing import TypedDict
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import requests

# 初始化模型和嵌入
# 使用 TinyLLaMA
model_name = "TinyLLaMA/TinyLLaMA-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", torch_dtype=torch.float16)

hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=200,  
    temperature=0.7,
    do_sample=True
)
llm = HuggingFacePipeline(pipeline=hf_pipeline)

# 使用開源嵌入模型
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Supabase 配置
SUPABASE_URL = 
SUPABASE_KEY = 
TABLE_NAME = "inventory"
headers = {
    "apikey": SUPABASE_KEY,
    "Authorization": f"Bearer {SUPABASE_KEY}",
    "Content-Type": "application/json"
}

# 初始化本地向量資料庫
vectorstore = None

# 定義狀態
class InventoryState(TypedDict):
    question: str
    retrieved_data: str
    answer: str

# 從 Supabase 獲取數據
def fetch_inventory_from_supabase():
    response = requests.get(f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}", headers=headers)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    else:
        return pd.DataFrame(columns=["飲料名稱", "進貨數量", "銷貨數量", "日期", "過期日期"])

# 將數據寫入 Supabase
def write_inventory_to_supabase(data):
    response = requests.post(
        f"{SUPABASE_URL}/rest/v1/{TABLE_NAME}",
        headers=headers,
        json=data
    )
    return response.status_code == 201

# 更新向量資料庫
def update_vectorstore(df):
    global vectorstore
    df["日期"] = pd.to_datetime(df["日期"])
    df["過期日期"] = pd.to_datetime(df["過期日期"], errors="coerce")
    documents = df.to_string(index=False)
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    texts = text_splitter.split_text(documents)
    vectorstore = Chroma.from_texts(texts, embeddings)

# 定義節點函數
def retrieve_node(state: InventoryState) -> InventoryState:
    retriever = vectorstore.as_retriever()
    retrieved_docs = retriever.get_relevant_documents(state["question"])
    state["retrieved_data"] = "\n".join([doc.page_content for doc in retrieved_docs])
    return state

def analyze_node(state: InventoryState) -> InventoryState:
    prompt = f"""
    根據以下進銷存數據，判斷是否需要進貨並檢查過期情況：
    - 數據：\n{state['retrieved_data']}
    - 規則：
      1. 如果有效庫存低於 50 瓶，建議進貨。
      2. 如果庫存維持天數低於 7 天，建議進貨。
      3. 如果有飲料即將在 30 天內過期，提醒優先處理。
      4. 避免進貨過多導致過期風險（參考每日銷貨量）。
    - 問題：{state['question']}
    請提供自然語言回應。
    """
    response = llm.invoke(prompt)
    state["answer"] = response
    return state

# 構建 LangGraph 工作流程
workflow = StateGraph(InventoryState)
workflow.add_node("retrieve", retrieve_node)
workflow.add_node("analyze", analyze_node)
workflow.add_edge(START, "retrieve")
workflow.add_edge("retrieve", "analyze")
workflow.add_edge("analyze", END)
app = workflow.compile()

# 處理數據輸入
def add_inventory(name, purchase_qty, sales_qty, date, expiry_date):
    new_data = {
        "飲料名稱": name,
        "進貨數量": float(purchase_qty),  # Supabase 需要明確類型
        "銷貨數量": float(sales_qty),
        "日期": date,
        "過期日期": expiry_date if purchase_qty > 0 else None
    }
    if write_inventory_to_supabase(new_data):
        df = fetch_inventory_from_supabase()
        update_vectorstore(df)
        return "數據已成功新增至線上資料庫！"
    return "新增失敗，請檢查網路或 API 配置。"

# 處理查詢
def query_inventory(question):
    if vectorstore is None:
        df = fetch_inventory_from_supabase()
        if df.empty:
            return "線上資料庫無數據，請先輸入數據！"
        update_vectorstore(df)
    result = app.invoke({"question": question, "retrieved_data": "", "answer": ""})
    return result["answer"]

# 創建 Gradio 介面
with gr.Blocks(title="飲料進銷存管理系統") as demo:
    gr.Markdown("### 飲料進銷存管理系統")

    with gr.Tab("輸入進銷存數據"):
        name_input = gr.Textbox(label="飲料名稱")
        purchase_input = gr.Number(label="進貨數量", value=0)
        sales_input = gr.Number(label="銷貨數量", value=0)
        date_input = gr.Textbox(label="日期 (YYYY-MM-DD)", value="2025-03-12")
        expiry_input = gr.Textbox(label="過期日期 (YYYY-MM-DD，若無進貨可留空)")
        submit_btn = gr.Button("提交數據")
        output = gr.Textbox(label="結果")
        submit_btn.click(
            fn=add_inventory,
            inputs=[name_input, purchase_input, sales_input, date_input, expiry_input],
            outputs=output
        )

    with gr.Tab("查詢進銷存"):
        question_input = gr.Textbox(label="輸入問題（例如：可樂需要進貨嗎？）")
        query_btn = gr.Button("查詢")
        query_output = gr.Textbox(label="回應")
        query_btn.click(
            fn=query_inventory,
            inputs=question_input,
            outputs=query_output
        )

# 啟動介面
demo.launch()

Device set to use cpu


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2984ba65ba6b84a6d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
!pip install langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.6/133.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.8 MB/s eta 0:00:00
